<a href="https://colab.research.google.com/github/DineshKatravath/da6401_assignment1/blob/main/DA6401_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connecting to Wandb

In [ ]:
!pip install wandb

In [1]:
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cs24m018 (cs24m018-iitmaana) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
import wandb

# Initialize a new WandB run
wandb.init(project="DA6401 Assignment1")

# Finish the run when you're done
wandb.finish()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cs24m018 (cs24m018-iitmaana) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


# Question 1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist

# Load the Fashion-MNIST dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()


wandb.init(project="DA6401 Assignment1")
runName = "Generating Images"
wandb.run.name = runName
wandb.run.save()

# Define class labels
class_labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

fig, axs = plt.subplots(2,5)
images = []

# Loop through each class
for i in range(10):
    # Find the first image of the current class in the training set
    idx = np.where(y_train == i)[0][0]

    # Plot the image
    x = i//5
    y = i%5

    wandb.log({"Image": wandb.Image(x_train[idx],caption=class_labels[i])})
    axs[x,y].imshow(x_train[idx], cmap='gray')
    axs[x,y].set_title(class_labels[i])
    axs[x,y].axis('off')  # Turn off axis for better visualization

wandb.finish()

plt.tight_layout()
plt.show()

# Question 2

In [2]:
from keras.datasets import mnist
from keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Activation Functions

def sigmoid(X):
  X = np.clip(X, -500, 500)  # Clipping x to avoid overflow
  return 1./(1+np.exp(-X))

def tanh(X):
  return np.tanh(X)

def relu(X):
  return np.maximum(0,X)

def softMax(X):
  # subtracting maxVal for stability
  maxVal = np.max(X,axis=0,keepdims=True)
  X = X - maxVal
  return np.exp(X)/np.sum(np.exp(X),axis=0,keepdims=True)


# Gradient Activation Functions

def sigmoidGrad(X):
  S = sigmoid(X)
  return S * (1 - S)

def tanhGrad(X):
  return 1-tanh(X)**2

def reluGrad(X):
  return np.where(X>0,1,0)

def softMaxGrad(X):
  S = softMax(X)
  return np.diagflat(S) - np.outer(S, S)

# Weight Initializations

def randomInit(shape):
  return np.random.randn(*shape)*0.01

def xavierInit(shape):
  inputCount,outputCount = shape
  std = np.sqrt(2/(inputCount+outputCount))
  return np.random.normal(0,std,shape)

In [23]:
class feedForwardNeuralNetwork:

  def __init__(self,inputSize,hiddenSize,hiddenLayerCount,outputSize,epochs,batchSize,optimiser,weightDecay,beta=0.9,lossFunc="crossEntropyLoss",activationFunc="tanh",learningRate=0.001,beta1=0.9,beta2=0.999,dataset="fashionMNIST",isWandb=False,initMode="xavier"):
    self.inputSize = inputSize
    self.hiddenSize = hiddenSize
    self.hiddenLayerCount = hiddenLayerCount
    self.outputSize = outputSize
    self.epochs = epochs
    self.batchSize = batchSize
    self.weightDecay = weightDecay
    self.beta = beta
    self.beta1 = beta1
    self.beta2 = beta2
    self.lossFunc = lossFunc
    self.activationFunc = activationFunc
    self.learningRate = learningRate
    self.optimiser = optimiser
    self.isWandb = isWandb
    self.initMode = initMode
    self.dataset = dataset

    self.weights = []
    self.biases = []
    self.activationLayer = []
    self.preActivationLayer = []

    # loading the data from dataset into X and y into train,test and val data
    if self.dataset == "fashionMNIST":
      (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
      X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)
    elif self.dataset == "MNIST":
      (X_train, y_train), (X_test, y_test) = mnist.load_data()
      X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

    # normalizing and resizing images
    X_train = X_train.flatten().reshape(X_train.shape[0], -1) / 255.0
    X_val = X_val.flatten().reshape(X_val.shape[0], -1) / 255.0
    X_test = X_test.flatten().reshape(X_test.shape[0], -1) / 255.0

    X_train = X_train.T
    X_val = X_val.T
    X_test = X_test.T

    self.X_train = X_train
    self.y_train = y_train
    self.X_val = X_val
    self.y_val = y_val
    self.X_test = X_test
    self.y_test = y_test

  def initialize(self):

    weights = []
    biases = []
    activationLayer = []
    preActivationLayer = []

    #initializing inputWeights
    if self.initMode == 'xavier':
      weights.append(xavierInit((self.hiddenSize,self.inputSize)))
    else:
      weights.append(randomInit((self.hiddenSize,self.inputSize)))
    biases.append(np.zeros((self.hiddenSize,1))) # initializing biases to 0.

    for i in range(1,self.hiddenLayerCount):
      if self.initMode == 'xavier': # initializing random weights
        weights.append(xavierInit((self.hiddenSize,self.hiddenSize)))
      else:
        weights.append(randomInit((self.hiddenSize,self.hiddenSize)))
      biases.append(np.zeros((self.hiddenSize,1))) # initializing biases to 0.

    # Initializing weights from lastHidden layer to output layer
    if self.initMode == 'xavier': # initializing random weights
      weights.append(xavierInit((self.outputSize,self.hiddenSize)))
    else:
      weights.append(randomInit((self.outputSize,self.hiddenSize)))
    biases.append(np.zeros((self.outputSize,1))) # initializing biases to 0.

    # updating parameters
    self.weights = weights
    self.biases = biases
    self.activationLayer = activationLayer
    self.preActivationLayer = preActivationLayer

  # takes a vector and applies activation function on it.
  def activateLayer(self,X):

    type = self.activationFunc
    if type == 'sigmoid':
      activatedLayer = sigmoid(X)
    elif type == 'tanh':
      activatedLayer = tanh(X)
    elif type == 'relu':
      activatedLayer = relu(X)
    elif type == 'softmax':
      activatedLayer = softMax(X)

    return activatedLayer

  # calculation g'(ai(x))
  def gradActivationLayer(self,X):

    type = self.activationFunc
    if type == 'sigmoid':
      gradActivatedLayer = sigmoidGrad(X)
    elif type == 'tanh':
      gradActivatedLayer = tanhGrad(X)
    elif type == 'relu':
      gradActivatedLayer = reluGrad(X)
    elif type == 'softmax':
      gradActivatedLayer = softMaxGrad(X)

    return gradActivatedLayer

  # applying softmax on output layer
  def outputActivationLayer(self,X):
    return softMax(X)

  # getting a0 = W0X+B
  def inputLayer(self,X):
    return np.dot(self.weights[0],X)+self.biases[0]

  # getting preactivation layers of next layer from activated layer of curr level and weights
  def hiddenLayer(self,X,layerNo):
    activatedLayer = self.activateLayer(X)
    self.activationLayer.append(activatedLayer)
    return np.dot(self.weights[layerNo],activatedLayer)+self.biases[layerNo]

  # forward propogation
  def forward(self,X):
    self.activationLayer=[]
    self.preActivationLayer=[]
    self.activationLayer.append(X) # h0 is input
    self.preActivationLayer.append(self.inputLayer(X))
    for num in range(1,self.hiddenLayerCount+1):
      self.preActivationLayer.append(self.hiddenLayer(self.preActivationLayer[num-1],num))

    self.output = self.outputActivationLayer(self.preActivationLayer[self.hiddenLayerCount])

    return self.output

  #backpropogation
  def backPropagation(self,X,ypred,y,weights,ai_x,hi_x,loss="crossEntropyLoss",activationType="softmax"):

    m = X.shape[1]
    dw = []
    db = []
    grad_ai = 0
    grad_al = 0
    grad_Wi = 0
    grad_bi = 0

    # gradient of loss w.r.t pre-activation of output layer

    y_one_hot = np.eye(self.outputSize)[y].T

    if loss == "crossEntropyLoss":
      grad_al = -(y_one_hot-ypred)
    elif loss == "squaredLoss":
      grad_al = (ypred-y_one_hot)*ypred*(1-ypred)
      grad_al = grad_al/m

    grad_ai = grad_al

    for k in range(self.hiddenLayerCount,-1,-1):
      # Gradient of loss w.r.t weights of kth layer
      grad_Wi = np.dot(grad_ai, hi_x[k].T)
      dw.append(grad_Wi)

      # Gradient of loss w.r.t biases of kth layer
      grad_bi = np.sum(grad_ai, axis=1, keepdims=True)
      db.append(grad_bi)

      if(k>0):
        grad_h = np.dot(weights[k].T,grad_ai)
        grad_ai = grad_h * self.gradActivationLayer(ai_x[k-1])

    # reversing arrays as we stored from last layer to first
    dw.reverse()
    db.reverse()

    for i in range(len(dw)):
      dw[i] += self.weightDecay*weights[i]

    return dw,db

  def calcAccuracyandLoss(self,weights,biases,input,labels,activationFunc="softmax"):
    n = input.shape[1]
    loss = 0
    accuracy = 0
    labels_one_hot = np.eye(10)[labels].T

    yPred = input
    for i in range(self.hiddenLayerCount):
      yPred = np.dot(weights[i],yPred)+biases[i]
      yPred = self.activateLayer(yPred)

    yPred = np.dot(weights[self.hiddenLayerCount],yPred)+biases[self.hiddenLayerCount]
    yPred = self.outputActivationLayer(yPred)

    for i in range(self.hiddenLayerCount+1):
      loss+= self.weightDecay*np.linalg.norm(weights[i])

    for i in range(n):
      y_pred = np.argmax(yPred[:,i])
      if labels[i]==y_pred:
        accuracy+=1

      if self.lossFunc == "crossEntropyLoss":
        loss += -np.log(yPred[:,i][labels[i]]+1e-15)
      else:
        loss += np.sum((yPred[:,i]-labels_one_hot[i])**2)

    return (accuracy*100)/n,loss/n

  def predict(self,weights,biases,input,labels):
    n = input.shape[1]
    loss = 0
    accuracy = 0

    yPred = input
    for i in range(self.hiddenLayerCount):
      yPred = np.dot(weights[i],yPred)+biases[i]
      yPred = self.activateLayer(yPred)

    yPred = np.dot(weights[self.hiddenLayerCount],yPred)+biases[self.hiddenLayerCount]
    yPred = self.outputActivationLayer(yPred)

    predictions = []
    for i in range(n):
      y_pred = np.argmax(yPred[:,i])
      predictions.append(y_pred)
    return predictions

  def sgd(self):
    self.initialize()

    it = 0
    while it < self.epochs:
      it += 1

      i = 0
      while i<len(self.X_train):
        batch_end = min(i + self.batchSize, self.X_train.shape[1])
        ypred = self.forward(self.X_train[:,i:batch_end])
        dw,db = self.backPropagation(self.X_train[:,i:batch_end],ypred,self.y_train[i:batch_end],self.weights,
                                     self.preActivationLayer,self.activationLayer,self.lossFunc,self.activationFunc)

        for k in range(len(dw)):
          self.weights[k] -= self.learningRate*dw[k]
          self.biases[k] -= self.learningRate*db[k]

        i += self.batchSize
      accuracy, loss = self.calcAccuracyandLoss(self.weights,self.biases,self.X_train,self.y_train,self.activationFunc)
      validationAccuracy , validationLoss = self.calcAccuracyandLoss(self.weights,self.biases,self.X_val,self.y_val,self.activationFunc)

      print("Validation Accuracy: ",validationAccuracy)
      print("Validation Loss: ",validationLoss)
      print("Training Accuracy: ",accuracy)
      print("Training Loss: ",loss)

      if self.isWandb:
        wandb.log({"Training Accuracy": accuracy})
        wandb.log({"Training Loss": loss})
        wandb.log({"Validation Accuracy": validationAccuracy})
        wandb.log({"Validation Loss": validationLoss})
        wandb.log({"Epoch": it})
        wandb.log({"Learning Rate": self.learningRate})
        wandb.log({"Batch Size": self.batchSize})
        wandb.log({"Optimiser": self.optimiser})

  def momentumBasedGradientDescent(self):
    self.initialize()

    # Properly initialize momentum
    uw = [np.zeros_like(w) for w in self.weights]
    ub = [np.zeros_like(b) for b in self.biases]

    it = 0
    while it < self.epochs:
      it += 1

      i = 0
      while i < len(self.X_train):
        batch_end = min(i + self.batchSize, self.X_train.shape[1])
        ypred = self.forward(self.X_train[:,i:batch_end])
        dw,db = self.backPropagation(self.X_train[:,i:batch_end],ypred,self.y_train[i:batch_end],self.weights,
                                     self.preActivationLayer,self.activationLayer,self.lossFunc,self.activationFunc)

        # Update momentum and weights for each layer
        for k in range(len(dw)):
          uw[k] = self.beta * uw[k] + dw[k]
          ub[k] = self.beta * ub[k] + db[k]

        for k in range(len(dw)):
          self.weights[k] -= self.learningRate * uw[k]
          self.biases[k] -= self.learningRate * ub[k]

        i += self.batchSize
      accuracy, loss = self.calcAccuracyandLoss(self.weights,self.biases,self.X_train,self.y_train,self.activationFunc)
      validationAccuracy , validationLoss = self.calcAccuracyandLoss(self.weights,self.biases,self.X_val,self.y_val,self.activationFunc)

      print("Validation Accuracy: ",validationAccuracy)
      print("Validation Loss: ",validationLoss)
      print("Training Accuracy: ",accuracy)
      print("Training Loss: ",loss)

      if self.isWandb:
        wandb.log({"Training Accuracy": accuracy})
        wandb.log({"Training Loss": loss})
        wandb.log({"Validation Accuracy": validationAccuracy})
        wandb.log({"Validation Loss": validationLoss})
        wandb.log({"Epoch": it})
        wandb.log({"Learning Rate": self.learningRate})
        wandb.log({"Batch Size": self.batchSize})
        wandb.log({"Optimiser": self.optimiser})

  def nesterovAcceleratedBasedGradientDescent(self):
    self.initialize()

    # Properly initialize momentum
    uw = [np.zeros_like(w) for w in self.weights]
    ub = [np.zeros_like(b) for b in self.biases]

    it=0
    while it<self.epochs:
      it+=1

      i=0
      while i<len(self.X_train):
        batch_end = min(i + self.batchSize, self.X_train.shape[1])
        ypred = self.forward(self.X_train[:,i:batch_end])
        dw,db = self.backPropagation(self.X_train[:,i:batch_end],ypred,self.y_train[i:batch_end],self.weights,
                                     self.preActivationLayer,self.activationLayer,self.lossFunc,self.activationFunc)

        for k in range(len(dw)):
          uw[k] = self.beta*uw[k] + dw[k]
          ub[k] = self.beta*ub[k] + db[k]

        for k in range(len(dw)):
          self.weights[k] -= self.learningRate*(self.beta*uw[k] + dw[k])
          self.biases[k] -= self.learningRate*(self.beta*ub[k] + db[k])

        i += self.batchSize
      accuracy, loss = self.calcAccuracyandLoss(self.weights,self.biases,self.X_train,self.y_train,self.activationFunc)
      validationAccuracy , validationLoss = self.calcAccuracyandLoss(self.weights,self.biases,self.X_val,self.y_val,self.activationFunc)

      print("Validation Accuracy: ",validationAccuracy)
      print("Validation Loss: ",validationLoss)
      print("Training Accuracy: ",accuracy)
      print("Training Loss: ",loss)

      if self.isWandb:
        wandb.log({"Training Accuracy": accuracy})
        wandb.log({"Training Loss": loss})
        wandb.log({"Validation Accuracy": validationAccuracy})
        wandb.log({"Validation Loss": validationLoss})
        wandb.log({"Epoch": it})
        wandb.log({"Learning Rate": self.learningRate})
        wandb.log({"Batch Size": self.batchSize})
        wandb.log({"Optimiser": self.optimiser})

  def rmsProp(self):
    self.initialize()

    v_w = [np.zeros_like(w) for w in self.weights]
    v_b = [np.zeros_like(b) for b in self.biases]

    eps = 1e-8
    it = 0

    while it < self.epochs:
      it += 1

      i = 0
      while i < len(self.X_train):
          batch_end = min(i + self.batchSize, self.X_train.shape[1])
          X_batch = self.X_train[:, i:batch_end]
          y_batch = self.y_train[i:batch_end]

          # Forward pass
          ypred = self.forward(X_batch)

          # Backward pass
          dw, db = self.backPropagation(X_batch, ypred, y_batch, self.weights,
                                      self.preActivationLayer, self.activationLayer,self.lossFunc, self.activationFunc)

          # Update velocity and weights for each layer
          for k in range(len(dw)):
              v_w[k] = self.beta * v_w[k] + (1 - self.beta) * (dw[k]**2)
              v_b[k] = self.beta * v_b[k] + (1 - self.beta) * (db[k]**2)

              self.weights[k] -= (self.learningRate / (np.sqrt(v_w[k]) + eps)) * dw[k]
              self.biases[k] -= (self.learningRate / (np.sqrt(v_b[k]) + eps)) * db[k]
          i += self.batchSize
      accuracy, loss = self.calcAccuracyandLoss(self.weights,self.biases,self.X_train,self.y_train,self.activationFunc)
      validationAccuracy , validationLoss = self.calcAccuracyandLoss(self.weights,self.biases,self.X_val,self.y_val,self.activationFunc)

      print("Validation Accuracy: ",validationAccuracy)
      print("Validation Loss: ",validationLoss)
      print("Training Accuracy: ",accuracy)
      print("Training Loss: ",loss)

      if self.isWandb:
        wandb.log({"Training Accuracy": accuracy})
        wandb.log({"Training Loss": loss})
        wandb.log({"Validation Accuracy": validationAccuracy})
        wandb.log({"Validation Loss": validationLoss})
        wandb.log({"Epoch": it})
        wandb.log({"Learning Rate": self.learningRate})
        wandb.log({"Batch Size": self.batchSize})
        wandb.log({"Optimiser": self.optimiser})

  def adam(self):
    self.initialize()

    v_w = [np.zeros_like(w) for w in self.weights]
    v_b = [np.zeros_like(b) for b in self.biases]
    m_w = [np.zeros_like(w) for w in self.weights]
    m_b = [np.zeros_like(b) for b in self.biases]

    eps = 1e-8
    it = 0
    t = 0

    while it < self.epochs:
      it += 1
      i = 0

      while i < len(self.X_train):
        batch_end = min(i + self.batchSize, self.X_train.shape[1])
        X_batch = self.X_train[:, i:batch_end]
        y_batch = self.y_train[i:batch_end]

        # Forward pass
        ypred = self.forward(X_batch)

        # Backward pass
        dw, db = self.backPropagation(X_batch, ypred, y_batch, self.weights,
                                    self.preActivationLayer, self.activationLayer, self.lossFunc, self.activationFunc)

        t += 1

        # Update momentum for each layer
        for k in range(len(dw)):
          m_w[k] = self.beta1 * m_w[k] + (1 - self.beta1) * dw[k]
          m_b[k] = self.beta1 * m_b[k] + (1 - self.beta1) * db[k]

          v_w[k] = self.beta2 * v_w[k] + (1 - self.beta2) * (dw[k]**2)
          v_b[k] = self.beta2 * v_b[k] + (1 - self.beta2) * (db[k]**2)

          # Bias correction
          mhat_w = m_w[k] / (1 - self.beta1**t)
          mhat_b = m_b[k] / (1 - self.beta1**t)

          vhat_w = v_w[k] / (1 - self.beta2**t)
          vhat_b = v_b[k] / (1 - self.beta2**t)

          self.weights[k] -= (self.learningRate / (np.sqrt(vhat_w) + eps)) * mhat_w
          self.biases[k] -= (self.learningRate / (np.sqrt(vhat_b) + eps)) * mhat_b
        i += self.batchSize
      accuracy, loss = self.calcAccuracyandLoss(self.weights,self.biases,self.X_train,self.y_train,self.activationFunc)
      validationAccuracy , validationLoss = self.calcAccuracyandLoss(self.weights,self.biases,self.X_val,self.y_val,self.activationFunc)

      print("Validation Accuracy: ",validationAccuracy)
      print("Validation Loss: ",validationLoss)
      print("Training Accuracy: ",accuracy)
      print("Training Loss: ",loss)

      if self.isWandb:
        wandb.log({"Training Accuracy": accuracy})
        wandb.log({"Training Loss": loss})
        wandb.log({"Validation Accuracy": validationAccuracy})
        wandb.log({"Validation Loss": validationLoss})
        wandb.log({"Epoch": it})
        wandb.log({"Learning Rate": self.learningRate})
        wandb.log({"Batch Size": self.batchSize})
        wandb.log({"Optimiser": self.optimiser})

  def nadam(self):
    self.initialize()

    v_w = [np.zeros_like(w) for w in self.weights]
    v_b = [np.zeros_like(b) for b in self.biases]
    m_w = [np.zeros_like(w) for w in self.weights]
    m_b = [np.zeros_like(b) for b in self.biases]

    eps = 1e-8
    it = 0
    t = 0

    while it < self.epochs:
      it += 1
      i = 0

      while i < self.X_train.shape[1]:
        batch_end = min(i + self.batchSize, self.X_train.shape[1])
        X_batch = self.X_train[:, i:batch_end]
        y_batch = self.y_train[i:batch_end]

        # Forward pass
        ypred = self.forward(X_batch)

        # Backward pass
        dw, db = self.backPropagation(X_batch, ypred, y_batch, self.weights,
                                    self.preActivationLayer, self.activationLayer, self.lossFunc, self.activationFunc)

        t += 1

        # Update momentum for each layer
        for k in range(len(dw)):
          m_w[k] = self.beta1 * m_w[k] + (1 - self.beta1) * dw[k]
          m_b[k] = self.beta1 * m_b[k] + (1 - self.beta1) * db[k]

          v_w[k] = self.beta2 * v_w[k] + (1 - self.beta2) * (dw[k]**2)
          v_b[k] = self.beta2 * v_b[k] + (1 - self.beta2) * (db[k]**2)

          # Bias correction
          mhat_w = m_w[k] / (1 - self.beta1**t)
          mhat_b = m_b[k] / (1 - self.beta1**t)

          vhat_w = v_w[k] / (1 - self.beta2**t)
          vhat_b = v_b[k] / (1 - self.beta2**t)

          self.weights[k] -= (self.learningRate / (np.sqrt(vhat_w) + eps)) * (self.beta1 * mhat_w + (1 - self.beta1) * dw[k] / (1 - self.beta1**t))
          self.biases[k] -= (self.learningRate / (np.sqrt(vhat_b) + eps)) * (self.beta1 * mhat_b + (1 - self.beta1) * db[k] / (1 - self.beta1**t))
        i += self.batchSize
      accuracy, loss = self.calcAccuracyandLoss(self.weights,self.biases,self.X_train,self.y_train,self.activationFunc)
      validationAccuracy , validationLoss = self.calcAccuracyandLoss(self.weights,self.biases,self.X_val,self.y_val,self.activationFunc)

      print("Validation Accuracy: ",validationAccuracy)
      print("Validation Loss: ",validationLoss)
      print("Training Accuracy: ",accuracy)
      print("Training Loss: ",loss)

      if self.isWandb:
        wandb.log({"Training Accuracy": accuracy})
        wandb.log({"Training Loss": loss})
        wandb.log({"Validation Accuracy": validationAccuracy})
        wandb.log({"Validation Loss": validationLoss})
        wandb.log({"Epoch": it})
        wandb.log({"Learning Rate": self.learningRate})
        wandb.log({"Batch Size": self.batchSize})
        wandb.log({"Optimiser": self.optimiser})

  def fit(self):
    if self.optimiser == "sgd":
      self.sgd()
    elif self.optimiser == "momentumBasedGradientDescent":
      self.momentumBasedGradientDescent()
    elif self.optimiser == "nesterovAcceleratedBasedGradientDescent":
      self.nesterovAcceleratedBasedGradientDescent()
    elif self.optimiser == "rmsProp":
      self.rmsProp()
    elif self.optimiser == "adam":
      self.adam()
    elif self.optimiser == "nadam":
      self.nadam()

  def confusionMatrix(self):
    predictions = self.predict(self.weights,self.biases,self.X_test,self.y_test)

    if self.dataset == "fashion_mnist":
      classLabels = ["T-shirt/Top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle Boot"]
    else:
      classLabels = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]

    if self.isWandb == True:
      conf_matrix = confusion_matrix(self.y_test, predictions)
      plt.figure(figsize=(10, 7))
      sns_heatmap = sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                                xticklabels=classLabels, yticklabels=classLabels)
      plt.title('Confusion Matrix')
      plt.ylabel('True Label')
      plt.xlabel('Predicted Label')

      # Save the plot to an image file
      heatmap_image_filename = "confusion_matrix_heatmap.png"
      plt.savefig(heatmap_image_filename)
      plt.close()  # Close the plot to avoid displaying it in the notebook/output

      # Log the image to Wandb
      wandb.log({"confusion_matrix_custom": wandb.Image(heatmap_image_filename)})

    else:
      conf_matrix = confusion_matrix(self.y_test, predictions)
      plt.figure(figsize=(10, 7))
      sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                  xticklabels= classLabels,
                  yticklabels= classLabels)
      plt.title('Confusion Matrix')
      plt.ylabel('True Label')
      plt.xlabel('Predicted Label')
      plt.show()

In [24]:
model = feedForwardNeuralNetwork(inputSize=784,hiddenSize=64,hiddenLayerCount=4,outputSize=10,batchSize=32,learningRate=1e-4,initMode = "xavier", optimiser = "nadam", activationFunc="tanh",weightDecay = 0.0005,lossFunc = "crossEntropyLoss", epochs = 25,dataset="fashionMNIST")

In [25]:
model.fit()

Validation Accuracy:  83.15833333333333
Validation Loss:  0.4960096356432739
Training Accuracy:  83.25833333333334
Training Loss:  0.49185336444688654
Validation Accuracy:  85.15833333333333
Validation Loss:  0.4278684353005466
Training Accuracy:  85.30833333333334
Training Loss:  0.4186025767083022
Validation Accuracy:  85.70833333333333
Validation Loss:  0.40207305812390487
Training Accuracy:  86.23125
Training Loss:  0.3878758139801917
Validation Accuracy:  86.225
Validation Loss:  0.3866299452307638
Training Accuracy:  86.91458333333334
Training Loss:  0.36815813996716623
Validation Accuracy:  86.55
Validation Loss:  0.37561930544232036
Training Accuracy:  87.40208333333334
Training Loss:  0.35330281106079026
Validation Accuracy:  86.875
Validation Loss:  0.36720654268858083
Training Accuracy:  87.8125
Training Loss:  0.3413445816619324
Validation Accuracy:  87.15833333333333
Validation Loss:  0.36041505451539674
Training Accuracy:  88.10833333333333
Training Loss:  0.3312309385803

# Question 4

In [ ]:
import wandb

sweep_config = {
    'method': 'random',  # Random search to sample configurations efficiently
    'metric': {
        'name': 'Validation Accuracy',  # Metric to optimize on the validation set
        'goal': 'maximize'
    },
    'parameters': {
        'epochs': {'values': [5, 10]},
        'hiddenLayerCount': {'values': [3, 4, 5]},
        'hiddenSize': {'values': [32, 64, 128]},
        'weightDecay': {'values': [0, 0.0005, 0.5]},
        'learningRate': {'values': [1e-3, 1e-4]},
        'optimiser': {'values': ['sgd', 'momentumBasedGradientDescent', 'nesterovAcceleratedBasedGradientDescent', 'rmsProp', 'adam', 'nadam']},
        'batchSize': {'values': [16, 32, 64]},
        'activationFunc': {'values': ['sigmoid', 'tanh', 'relu']},
    }
}

def main():
  wandb.init()
  config = wandb.config

  runName = f"hS{config.hiddenSize}_lr{config.learningRate}_{config.optimiser}_bs{config.batchSize}_af{config.activationFunc}"

  wandb.run.name = runName #setting wandb run
  wandb.run.save()

  model = feedForwardNeuralNetwork(inputSize=784,hiddenSize=config.hiddenSize,hiddenLayerCount=4,outputSize=10,batchSize=config.batchSize,
                                   learningRate=config.learningRate,initMode = 'xavier', optimiser = config.optimiser, activationFunc=config.activationFunc,weightDecay = 0.0005,
                                   epochs = 10,isWandb=True,lossFunc='crossEntropyLoss')
  model.fit()
  wandb.finish()

# create Sweep and running it
sweepId = wandb.sweep(sweep_config, project="DA6401 Assignment1")
wandb.agent(sweepId,main,count=10)


Create sweep with ID: 641wtoy1
Sweep URL: https://wandb.ai/cs24m018-iitmaana/DA6401%20Assignment1/sweeps/641wtoy1


wandb: Agent Starting Run: jhklt5bp with config:
wandb: 	activationFunc: relu
wandb: 	batchSize: 64
wandb: 	epochs: 5
wandb: 	hiddenLayerCount: 3
wandb: 	hiddenSize: 128
wandb: 	initMode: random
wandb: 	learningRate: 0.0001
wandb: 	lossFunc: crossEntropyLoss
wandb: 	optimiser: nesterovAcceleratedBasedGradientDescent
wandb: 	weightDecay: 0.0005


Validation Accuracy:  27.683333333333334
Validation Loss:  2.151340533725651
Training Accuracy:  27.63125
Training Loss:  2.1525845709126386
Validation Accuracy:  47.858333333333334
Validation Loss:  1.8419491238086954
Training Accuracy:  47.87916666666667
Training Loss:  1.8432237049147864
Validation Accuracy:  56.425
Validation Loss:  1.4230194572196848
Training Accuracy:  56.46666666666667
Training Loss:  1.425251078989168
Validation Accuracy:  63.391666666666666
Validation Loss:  1.1110074166149975
Training Accuracy:  63.208333333333336
Training Loss:  1.1138068085381725
Validation Accuracy:  67.25
Validation Loss:  0.927928558976892
Training Accuracy:  66.60625
Training Loss:  0.9315489580890348
Validation Accuracy:  70.16666666666667
Validation Loss:  0.8358184662988415
Training Accuracy:  69.63125
Training Loss:  0.8403370898075526
Validation Accuracy:  72.15833333333333
Validation Loss:  0.789848149308957
Training Accuracy:  71.87291666666667
Training Loss:  0.7941835914490541


Batch Size,▁▁▁▁▁▁▁▁▁▁
Epoch,▁▂▃▃▄▅▆▆▇█
Learning Rate,▁▁▁▁▁▁▁▁▁▁
Training Accuracy,▁▄▅▆▇▇▇███
Training Loss,█▆▄▃▂▂▁▁▁▁
Validation Accuracy,▁▄▅▆▇▇▇███
Validation Loss,█▆▄▃▂▂▁▁▁▁
Batch Size,64
Epoch,10
Learning Rate,0.0001
Optimiser,nesterovAcceleratedB...


wandb: Agent Starting Run: boi0t5y4 with config:
wandb: 	activationFunc: sigmoid
wandb: 	batchSize: 32
wandb: 	epochs: 10
wandb: 	hiddenLayerCount: 4
wandb: 	hiddenSize: 32
wandb: 	initMode: xavier
wandb: 	learningRate: 0.0001
wandb: 	lossFunc: squaredLoss
wandb: 	optimiser: nadam
wandb: 	weightDecay: 0.0005


...

Validation Accuracy:  41.983333333333334
Validation Loss:  1.9542796421008055
Training Accuracy:  41.62916666666667
Training Loss:  1.9567340059014295
Validation Accuracy:  53.075
Validation Loss:  1.7985950937567468
Training Accuracy:  52.56666666666667
Training Loss:  1.8019976664797281
Validation Accuracy:  58.525
Validation Loss:  1.7110324191990594
Training Accuracy:  58.047916666666666
Training Loss:  1.7146274943233315
Validation Accuracy:  62.74166666666667
Validation Loss:  1.659690394574905
Training Accuracy:  62.09791666666667
Training Loss:  1.662944563961957
Validation Accuracy:  65.40833333333333
Validation Loss:  1.6271276545595137
Training Accuracy:  64.76875
Training Loss:  1.6299784242447561
Validation Accuracy:  66.98333333333333
Validation Loss:  1.6046876661762755
Training Accuracy:  66.49375
Training Loss:  1.6071786987483647
Validation Accuracy:  68.25833333333334
Validation Loss:  1.587961420955138
Training Accuracy:  67.68125
Training Loss:  1.590155300332562
V

Batch Size,▁▁▁▁▁▁▁▁▁▁
Epoch,▁▂▃▃▄▅▆▆▇█
Learning Rate,▁▁▁▁▁▁▁▁▁▁
Training Accuracy,▁▄▅▆▇▇▇▇██
Training Loss,█▅▄▃▂▂▂▁▁▁
Validation Accuracy,▁▄▅▆▇▇▇▇██
Validation Loss,█▅▄▃▂▂▂▁▁▁
Batch Size,32
Epoch,10
Learning Rate,0.0001
Optimiser,adam


wandb: Agent Starting Run: tak629fv with config:
wandb: 	activationFunc: tanh
wandb: 	batchSize: 16
wandb: 	epochs: 5
wandb: 	hiddenLayerCount: 5
wandb: 	hiddenSize: 128
wandb: 	initMode: xavier
wandb: 	learningRate: 0.001
wandb: 	lossFunc: squaredLoss
wandb: 	optimiser: adam
wandb: 	weightDecay: 0


Validation Accuracy:  69.43333333333334
Validation Loss:  1.5558725512957448
Training Accuracy:  69.975
Training Loss:  1.5533287190780214
Validation Accuracy:  73.625
Validation Loss:  1.5345524054378537
Training Accuracy:  73.63125
Training Loss:  1.5313939187212258
Validation Accuracy:  76.28333333333333
Validation Loss:  1.535183123354323
Training Accuracy:  76.40208333333334
Training Loss:  1.5328621835586573
Validation Accuracy:  75.80833333333334
Validation Loss:  1.5609157607687036
Training Accuracy:  76.59583333333333
Training Loss:  1.5598483012453312
Validation Accuracy:  78.63333333333334
Validation Loss:  1.5367711528354244
Training Accuracy:  79.15208333333334
Training Loss:  1.5363312415157797
Validation Accuracy:  77.70833333333333
Validation Loss:  1.535332551549001
Training Accuracy:  77.9875
Training Loss:  1.5360318128113046
Validation Accuracy:  77.7
Validation Loss:  1.512833065051321
Training Accuracy:  77.91875
Training Loss:  1.5134541354984774
Validation Accur

Batch Size,▁▁▁▁▁▁▁▁▁▁
Epoch,▁▂▃▃▄▅▆▆▇█
Learning Rate,▁▁▁▁▁▁▁▁▁▁
Training Accuracy,▁▄▆▆█▇▇▆▆▆
Training Loss,▅▃▃▆▃▃▁▂▄█
Validation Accuracy,▁▄▆▆█▇▇▆▇▆
Validation Loss,▆▃▃▆▄▃▁▃▄█
Batch Size,16
Epoch,10
Learning Rate,0.001
Optimiser,adam


In [ ]:
# for printing the confusion matrix
import wandb

# Define the sweep configuration
sweep_config = {
    'method': 'bayes',
    'name' : 'confusion matrix',
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'
    },
    'parameters': {
        'batchSize': {
            'values': [64]
        },
        # Define other parameters here
    }
}
def main():
    # Initialize a wandb run
    wandb.init()
    # Access sweep parameters via wandb.config
    config = wandb.config

    # Define and train the model using parameters from config
    model = feedForwardNeuralNetwork(inputSize=784, hiddenLayerCount=4, outputSize=10,
                          hiddenSize=128, batchSize=config.batchSize,
                          learningRate=0.0001, initMode="xavier",
                          optimiser="momentumBasedGradientDescent", activationFunc="tanh", weightDecay=0.0005,
                          epochs=10, isWandb=True)
    model.fit()
    model.confusionMatrix()

    wandb.finish()

# Create the sweep
sweep_id = wandb.sweep(sweep=sweep_config, project='DA6401 Assignment1')

# Start the sweep agent
wandb.agent(sweep_id, main, count=1)

Create sweep with ID: aaaem2r8
Sweep URL: https://wandb.ai/cs24m018-iitmaana/DA6401%20Assignment1/sweeps/aaaem2r8


wandb: Agent Starting Run: n1n7enl8 with config:
wandb: 	batchSize: 64


Validation Accuracy:  43.675
Validation Loss:  1.663641675047539
Training Accuracy:  42.920833333333334
Training Loss:  1.6681291229315165
Validation Accuracy:  62.516666666666666
Validation Loss:  1.1891319818388566
Training Accuracy:  62.09166666666667
Training Loss:  1.1905778266728995
Validation Accuracy:  67.83333333333333
Validation Loss:  0.9573516957787567
Training Accuracy:  67.37083333333334
Training Loss:  0.9582262646376851
Validation Accuracy:  71.04166666666667
Validation Loss:  0.832575576754897
Training Accuracy:  70.54583333333333
Training Loss:  0.8317355078431924
Validation Accuracy:  73.35833333333333
Validation Loss:  0.761656766908565
Training Accuracy:  72.97708333333334
Training Loss:  0.7603746126886782
Validation Accuracy:  74.7
Validation Loss:  0.7174708474688547
Training Accuracy:  74.59375
Training Loss:  0.7153277084929078
Validation Accuracy:  75.69166666666666
Validation Loss:  0.6875545299269973
Training Accuracy:  75.45625
Training Loss:  0.6845986198

Batch Size,▁▁▁▁▁▁▁▁▁▁
Epoch,▁▂▃▃▄▅▆▆▇█
Learning Rate,▁▁▁▁▁▁▁▁▁▁
Training Accuracy,▁▅▆▇▇▇████
Training Loss,█▅▃▂▂▂▁▁▁▁
Validation Accuracy,▁▅▆▇▇▇████
Validation Loss,█▅▃▂▂▂▁▁▁▁
Batch Size,64
Epoch,10
Learning Rate,0.0001
Optimiser,momentumBasedGradien...


In [7]:
import wandb

sweep_config = {
    'method': 'bayes',  # Random search to sample configurations efficiently
    'name' : 'Cross Entropy Sweep',
    'metric': {
        'name': 'Validation Accuracy',  # Metric to optimize on the validation set
        'goal': 'maximize'
    },
    'parameters': {
        'epochs': {'values': [5, 10]},
        'hiddenLayerCount': {'values': [3, 4, 5]},
        'hiddenSize': {'values': [32, 64, 128]},
        'weightDecay': {'values': [0, 0.0005, 0.5]},
        'learningRate': {'values': [1e-2,1e-3, 1e-4]},
        'optimiser': {'values': ['sgd', 'momentumBasedGradientDescent', 'nesterovAcceleratedBasedGradientDescent', 'rmsProp', 'adam', 'nadam']},
        'batchSize': {'values': [16, 32, 64]},
        'initMode': {'values': ['random', 'xavier']},
        'activationFunc': {'values': ['sigmoid', 'tanh', 'relu']},
    }
}

def main():
  wandb.init()
  config = wandb.config

  runName = f"hS{config.hiddenSize}_lr{config.learningRate}_{config.optimiser}_bs{config.batchSize}_af{config.activationFunc}"

  wandb.run.name = runName #setting wandb run
  wandb.run.save()

  model = feedForwardNeuralNetwork(inputSize=784,hiddenSize=config.hiddenSize,hiddenLayerCount=config.hiddenSize,outputSize=10,batchSize=config.batchSize,
                                   learningRate=config.learningRate,initMode = config.initMode , optimiser = config.optimiser, activationFunc=config.activationFunc,weightDecay = config.weightDecay,
                                   epochs = config.epochs,isWandb=True,lossFunc='crossEntropyLoss')
  model.fit()
  wandb.finish()

# create Sweep and running it
sweepId = wandb.sweep(sweep_config, project="DA6401 Assignment1")
wandb.agent(sweep_id=sweepId, function=main, count=100)

Create sweep with ID: 292018fh
Sweep URL: https://wandb.ai/cs24m018-iitmaana/DA6401%20Assignment1/sweeps/292018fh


wandb: Agent Starting Run: z7fh4ngq with config:
wandb: 	activationFunc: sigmoid
wandb: 	batchSize: 64
wandb: 	epochs: 10
wandb: 	hiddenLayerCount: 3
wandb: 	hiddenSize: 128
wandb: 	initMode: xavier
wandb: 	learningRate: 0.001
wandb: 	optimiser: nesterovAcceleratedBasedGradientDescent
wandb: 	weightDecay: 0.0005


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Validation Accuracy:  10.116666666666667
Validation Loss:  2.3376726835931194
Training Accuracy:  9.970833333333333
Training Loss:  2.3377776320601846
Validation Accuracy:  10.116666666666667
Validation Loss:  2.325913886208041
Training Accuracy:  9.970833333333333
Training Loss:  2.325452497729105
Validation Accuracy:  10.116666666666667
Validation Loss:  2.325472138199385
Training Accuracy:  9.970833333333333
Training Loss:  2.3253365331200353
Validation Accuracy:  10.116666666666667
Validation Loss:  2.3258903296614397
Training Accuracy:  9.970833333333333
Training Loss:  2.326231303287146
Validation Accuracy:  10.116666666666667
Validation Loss:  2.3253679758704373
Training Accuracy:  9.970833333333333
Training Loss:  2.326135854470894
Validation Accuracy:  10.45
Validation Loss:  2.323115532320516
Training Accuracy:  9.8875
Training Loss:  2.3241425719009707
Validation Accuracy:  10.45
Validation Loss:  2.319704924682238
Training Accuracy:  9.8875
Training Loss:  2.320782503895661

Batch Size,▁▁▁▁▁▁▁▁▁▁
Epoch,▁▂▃▃▄▅▆▆▇█
Learning Rate,▁▁▁▁▁▁▁▁▁▁
Training Accuracy,█████▁▁▁▁▁
Training Loss,█▅▄▅▅▄▃▂▂▁
Validation Accuracy,▁▁▁▁▁█████
Validation Loss,█▅▅▅▅▄▃▂▁▁
Batch Size,64
Epoch,10
Learning Rate,0.001
Optimiser,nesterovAcceleratedB...


...

Validation Accuracy:  9.975
Validation Loss:  2.302585196689991
Training Accuracy:  10.00625
Training Loss:  2.302589226988604
Validation Accuracy:  9.975
Validation Loss:  2.3025848028661677
Training Accuracy:  10.00625
Training Loss:  2.3025923440000557
Validation Accuracy:  9.975
Validation Loss:  2.3025905030211087
Training Accuracy:  10.00625
Training Loss:  2.302601756697289
Validation Accuracy:  9.975
Validation Loss:  2.3026058108466976
Training Accuracy:  10.00625
Training Loss:  2.3026214173822583
Validation Accuracy:  9.975
Validation Loss:  2.302639435305601
Training Accuracy:  10.00625
Training Loss:  2.3026604903628276


Batch Size,▁▁▁▁▁
Epoch,▁▃▅▆█
Learning Rate,▁▁▁▁▁
Training Accuracy,▁▁▁▁▁
Training Loss,▁▁▂▄█
Validation Accuracy,▁▁▁▁▁
Validation Loss,▁▁▂▄█
Batch Size,64
Epoch,5
Learning Rate,0.001
Optimiser,adam


wandb: Agent Starting Run: bjg6bo2y with config:
wandb: 	activationFunc: sigmoid
wandb: 	batchSize: 64
wandb: 	epochs: 10
wandb: 	hiddenLayerCount: 3
wandb: 	hiddenSize: 128
wandb: 	initMode: xavier
wandb: 	learningRate: 0.001
wandb: 	optimiser: rmsProp
wandb: 	weightDecay: 0.5


Validation Accuracy:  9.825
Validation Loss:  2.377128439332326
Training Accuracy:  10.04375
Training Loss:  2.336250834110984
Validation Accuracy:  9.825
Validation Loss:  2.3706578015038335
Training Accuracy:  10.04375
Training Loss:  2.3344119191933443
Validation Accuracy:  9.825
Validation Loss:  2.3652737734363556
Training Accuracy:  10.04375
Training Loss:  2.332980240360627
Validation Accuracy:  9.825
Validation Loss:  2.3604314253728997
Training Accuracy:  10.04375
Training Loss:  2.331714762701755
Validation Accuracy:  9.825
Validation Loss:  2.3560124927001533
Training Accuracy:  10.04375
Training Loss:  2.3305571434840466


wandb: Ctrl + C detected. Stopping sweep.
